In [1]:
import json
import copy
import argparse
import re
import json
from datetime import datetime
from collections import Counter
import sys
import time
from openai import OpenAI

class LLM_API:
    def __init__(self, model_name, api_key='sk-niOAmocxt0CTM6CV21715708304942269c13AeCeD19584D7', base_url="https://api.claudeshop.top/v1"):
        self.client = OpenAI(api_key=api_key, base_url=base_url)
        self.model = model_name

    def respond(self,  messages, temperature=0.7, max_tokens=256, stop=None):

        repeat_num = 0
        response_data = None
        while response_data == None:
            repeat_num += 1
            if repeat_num>5:
                response_data = "I Don't Know!"
            try:
                completion = self.client.chat.completions.create(
                    model=self.model,
                    messages= messages,
                    timeout=15,
                    temperature=temperature,
                    max_tokens=max_tokens,
                    top_p=1,
                    stop=stop
                )
                response_data = completion.choices[0].message.content
                break
            except KeyboardInterrupt:
                sys.exit()
            except:
                print("Request timed out, retrying...")
                
        return response_data

llm_model = LLM_API("gpt-3.5-turbo")

In [2]:
grained_decompos = {}
grained_decompos["Achievement"] = ["\n1. Competence:\n- Positive Indicator: Emphasizes the importance of demonstrating skill and proficiency in tasks (e.g., excelling in academic or professional settings).\n- Negative Indicator: Shows indifference towards skill development or a lack of effort in demonstrating competence.",
                                   "\n2. Ambition:\n- Positive Indicator: Reflects a strong desire to achieve and excel (e.g., setting high goals and striving to reach them).\n- Negative Indicator: Indicates complacency or a lack of motivation to pursue personal success.",
                                   "\n3. Success:\n- Positive Indicator: Values tangible accomplishments and recognition for achievements (e.g., receiving awards or promotions).\n- Negative Indicator: Downplays the significance of success or equates it with superficial achievements.",
                                   "\n4. Social Approval:\n Positive Indicator: Acknowledges the importance of gaining recognition and respect from others (e.g., seeking validation from peers and society).\n - Negative Indicator: Dismisses the need for social recognition, prioritizing personal satisfaction over external validation."]

values_decrip = {}
values_decrip['Achievement'] = "Creativity is the value that drives the generation of novel and valuable ideas, solutions, or products through original thinking, embodying characteristics such as innovation, uniqueness, and societal or personal utility."

In [3]:
import pandas as pd
import json

# 打开JSON文件并读取数据
with open('Datasets/origin_qa/achievement_QA.json', 'r', encoding='utf-8') as file:
    qa_data = json.load(file)
with open("Datasets/prompt/exp3_rewrite_respond.txt", 'r') as file:
    rewrite_prompt_format = file.read()
with open("Datasets/prompt/exp3_rewrire_evaluate.txt", 'r') as file:
    evaluate_prompt_format = file.read()

In [23]:
import re
from src.role_person_generation import generate_persona_description
from src.role_occupation_generation import generate_persona_occupation_description

max_rewrite_num = 3
role_num = 4

values = "Achievement"
agree_level = "Strongly agree"
agree_score_base = 2
grained_decrip = ''.join(grained_decompos[values])
answer_pattern = r'(?<="New_Answer": ")[^"]*"'
evaluate_pattern = r'(?<="levels": ")[^"]*"'
for idx, qa in enumerate(qa_data):
    print("#########################",idx,"/",len(qa_data),"###########################################################")
    best_score = 999
    best_answer = None
    for j in range(max_rewrite_num):
        #1.重写回答
        question = qa['question']
        answer = qa['answer']
        print("#########################New Rewrite Answer###########################################################")
        print("Question:", question)
        print("Answer:", answer)
        rewrite_prompt = rewrite_prompt_format.replace("{value}", values)
        rewrite_prompt = rewrite_prompt.replace("{description}", values_decrip[values])
        rewrite_prompt = rewrite_prompt.replace("{grained_description}", grained_decrip)
        rewrite_prompt = rewrite_prompt.replace("{agree_level}", agree_level)
        rewrite_prompt = rewrite_prompt.replace("{question}", question)
        rewrite_prompt = rewrite_prompt.replace("{answer}", answer)

        messages = []
        messages.append({"role": "user", "content": rewrite_prompt})
        new_answer = llm_model.respond(messages)

        match = re.search(answer_pattern, new_answer)
        if match:
            new_answer = match.group(0)
        print("New_Answer:", new_answer)
        print("#########################Start person Evaluate###########################################################")
        #2.生成k个角色，分别对细粒度价值观进行评审
        personas = generate_persona_occupation_description(role_num)
        evaluate_scores = [0]*len(grained_decompos["Achievement"])
        for per in personas:
            person_prompt ="USER: "
            person_prompt += "Let's role-play. I will ask you a question and you must give me an answer. I want you to act as the person described below. Think from the person's perspective. \n"
            person_prompt += f"{per['description']} \n"
            person_prompt += "Use the given information to answer the question below. \n"

            for grain_idx, grained in enumerate(grained_decompos["Achievement"]):
                grained_person_prompt = person_prompt + evaluate_prompt_format.replace("{value}", values)
                grained_person_prompt = grained_person_prompt.replace("{description}", values_decrip[values])
                grained_person_prompt = grained_person_prompt.replace("{grained_description}", grained)
                grained_person_prompt = grained_person_prompt.replace("{statement}", new_answer)

                messages = []
                messages.append({"role": "user", "content": grained_person_prompt})
                evaluate_response = llm_model.respond(messages)

                if "agree" in evaluate_response.lower():
                    evaluate_score = 1
                elif "neutral" in evaluate_response.lower():
                    evaluate_score = 0
                elif "disagree" in evaluate_response.lower():
                    evaluate_score = -1
                else:
                    evaluate_score = 0
                evaluate_scores[grain_idx] += evaluate_score 
        #3.当分数差距小于0.3时，跳出
        agree_score = 2 * sum(evaluate_scores)/ (role_num) / len(evaluate_scores)
        if abs(agree_score-agree_score_base)<abs(best_score-agree_score_base):
            best_score = agree_score
            best_answer = new_answer
        print("Evaluate_score:", evaluate_score)
        print("Finally_score:", agree_score)
        print("Best_score:", best_score)
        print("Best_Answer:", best_answer)
        
        if abs(best_score-agree_score_base)<0.3:
            break
    print("Fianlly_Best_Answer:", best_answer)            
    break

######################### 0 / 854 ###########################################################
#########################New Rewrite Answer###########################################################
Question: What do you have planned for tomorrow that involves showcasing your singing talent?
Answer: Tomorrow I will audition my singing in a talent show.
New_Answer: I have a performance scheduled tomorrow where I will be showcasing my singing talent."
#########################Start person Evaluate###########################################################
Evaluate_score: 1
Finally_score: 1.125
Best_score: 1.125
Best_Answer: I have a performance scheduled tomorrow where I will be showcasing my singing talent."
#########################New Rewrite Answer###########################################################
Question: What do you have planned for tomorrow that involves showcasing your singing talent?
Answer: Tomorrow I will audition my singing in a talent show.
New_Answer: Tomorrow, I ha

In [22]:
2 * sum(evaluate_scores)/ (role_num) / len(evaluate_scores)

1.375

In [21]:
evaluate_scores

[1, 4, 3, 3]

In [11]:
evaluate_score = [0]*len(grained_decompos["Achievement"])

In [15]:
evaluate_score[grain_idx]

0

In [14]:
grain_idx

0